Katin's contribution to 'Create a blog post series: "Fundamentals of Data Visualization in Bokeh"': Explore NYC taxi trips dataset 6

In [1]:
#! pip install pyarrow

In [2]:
import pyarrow.parquet as pq
import pandas as pd
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.layouts import row, grid, layout, gridplot, Spacer
from bokeh.models import NumeralTickFormatter

I am curious to see trends over time. First, Load all the November datasets into dataframes. Find the type of data (from yellow or green taxi or high-volume/for hire trip types) that has the fewest number of rows, so that I can expand to a full year without creating a dataframe that is too large. 

In [3]:
nov22yellow = pq.read_table('./data/yellow_tripdata_2022-11.parquet')
nov22yellow = nov22yellow.to_pandas()
len(nov22yellow)

3252717

In [4]:
nov22green = pq.read_table('./data/green_tripdata_2022-11.parquet')
nov22green = nov22green.to_pandas()
len(nov22green)

62313

In [5]:
nov22fhvhv = pq.read_table('./data/fhvhv_tripdata_2022-11.parquet')
nov22fhvhv = nov22fhvhv.to_pandas()
len(nov22fhvhv)

18085896

In [6]:
nov22fhv = pq.read_table('./data/fhv_tripdata_2022-11.parquet')
nov22fhv = nov22fhv.to_pandas()
len(nov22fhv)

1106084

Green taxis data contains the fewest rows, so I'll collect all of the green taxi data for 2021 (since that's the first full year) into a dataframe.  

In [7]:
# Load each month.
jan21 = pq.read_table('./data/green_tripdata_2021-01.parquet')
jan21 = jan21.to_pandas()
feb21 = pq.read_table('./data/green_tripdata_2021-02.parquet')
feb21 = feb21.to_pandas()
mar21 = pq.read_table('./data/green_tripdata_2021-03.parquet')
mar21 = mar21.to_pandas()
apr21 = pq.read_table('./data/green_tripdata_2021-04.parquet')
apr21 = apr21.to_pandas()
may21 = pq.read_table('./data/green_tripdata_2021-05.parquet')
may21 = may21.to_pandas()
jun21 = pq.read_table('./data/green_tripdata_2021-06.parquet')
jun21 = jun21.to_pandas()
jul21 = pq.read_table('./data/green_tripdata_2021-07.parquet')
jul21 = jul21.to_pandas()
aug21 = pq.read_table('./data/green_tripdata_2021-08.parquet')
aug21 = aug21.to_pandas()
sep21 = pq.read_table('./data/green_tripdata_2021-09.parquet')
sep21 = sep21.to_pandas()
oct21 = pq.read_table('./data/green_tripdata_2021-10.parquet')
oct21 = oct21.to_pandas()
nov21 = pq.read_table('./data/green_tripdata_2021-11.parquet')
nov21 = nov21.to_pandas()
dec21 = pq.read_table('./data/green_tripdata_2021-12.parquet')
dec21 = dec21.to_pandas()

# concatenate into one df.
full21_df = pd.concat([jan21, feb21, mar21, apr21, may21, jun21, jul21, aug21, sep21, oct21,
                      nov21, dec21])
full21_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1068755 entries, 0 to 99960
Data columns (total 20 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   VendorID               1068755 non-null  int64         
 1   lpep_pickup_datetime   1068755 non-null  datetime64[ns]
 2   lpep_dropoff_datetime  1068755 non-null  datetime64[ns]
 3   store_and_fwd_flag     656321 non-null   object        
 4   RatecodeID             656321 non-null   float64       
 5   PULocationID           1068755 non-null  int64         
 6   DOLocationID           1068755 non-null  int64         
 7   passenger_count        656321 non-null   float64       
 8   trip_distance          1068755 non-null  float64       
 9   fare_amount            1068755 non-null  float64       
 10  extra                  1068755 non-null  float64       
 11  mta_tax                1068755 non-null  float64       
 12  tip_amount             1068755

What does a typical month look like? Or, what month/months hover around the average or median for total volume of rides or fare amounts?

In [8]:
# Prepare to analyze by month. 
full21_df['month'] = full21_df.lpep_pickup_datetime.dt.month
full21_df.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,month
0,2,2021-01-01 00:15:56,2021-01-01 00:19:52,N,1.0,43,151,1.0,1.01,5.5,...,0.5,0.00,0.0,None,0.3,6.80,2.0,1.0,0.00,1
1,2,2021-01-01 00:25:59,2021-01-01 00:34:44,N,1.0,166,239,1.0,2.53,10.0,...,0.5,2.81,0.0,None,0.3,16.86,1.0,1.0,2.75,1
2,2,2021-01-01 00:45:57,2021-01-01 00:51:55,N,1.0,41,42,1.0,1.12,6.0,...,0.5,1.00,0.0,None,0.3,8.30,1.0,1.0,0.00,1
3,2,2020-12-31 23:57:51,2021-01-01 00:04:56,N,1.0,168,75,1.0,1.99,8.0,...,0.5,0.00,0.0,None,0.3,9.30,2.0,1.0,0.00,12
4,2,2021-01-01 00:16:36,2021-01-01 00:16:40,N,2.0,265,265,3.0,0.00,-52.0,...,-0.5,0.00,0.0,None,-0.3,-52.80,3.0,1.0,0.00,1


In [9]:
# Group by month with ride count per month.
count_by_month = pd.DataFrame(full21_df.groupby('month')['lpep_pickup_datetime'].count())

# Rename column for clarity. 
count_by_month = count_by_month.rename(columns={'lpep_pickup_datetime':'ride_count'})

# Reset index for easier access
count_by_month = count_by_month.reset_index(drop=True)
count_by_month

,ride_count
0,76536
1,64570
2,83831
3,86931
4,88183
5,86747
6,83675
7,83491
8,95711
9,110891


In [10]:
# Do the same for total fares. 
fare_by_month = pd.DataFrame(full21_df.groupby('month')['fare_amount'].sum())
fare_by_month = fare_by_month.reset_index(drop=True)
fare_by_month

,fare_amount
0,1525899.37
1,1275508.76
2,1685423.04
3,1776339.48
4,1777846.24
5,1725091.92
6,1705742.60
7,1717961.52
8,2024972.78
9,2227963.69


In [11]:
# Plot both of these series next to each other

# Prepare figure for volume
count_fig = figure(title="Green Taxi 2021 Ride Volume", 
                   x_axis_label="month", y_axis_label="number of rides in thousands", 
                   width=400, height=400)

count_fig.line(x=count_by_month.index, y=(count_by_month.ride_count/1000),
              line_width=3, line_color="purple")

# Customize figure labels for easier reading
count_fig.title.text_font_size = "20px"
count_fig.title.align = "center"
count_fig.xaxis.axis_label_text_font_size = "15px"
count_fig.yaxis.axis_label_text_font_size = "15px"
count_fig.xaxis.major_label_text_font_size = "15px"
count_fig.yaxis.major_label_text_font_size = "15px"

# Repeat for fare
fare_fig = figure(title="Green Taxi 2021 Fares", 
                  x_axis_label="month", y_axis_label="total fare revenue in thousands", 
                  width=400, height=400)

fare_fig.line(x=fare_by_month.index, y=(fare_by_month.fare_amount/1000), 
             line_width=3, line_color="purple")

fare_fig.title.text_font_size = "20px"
fare_fig.title.align = "center"
fare_fig.xaxis.axis_label_text_font_size = "15px"
fare_fig.yaxis.axis_label_text_font_size = "15px"
fare_fig.xaxis.major_label_text_font_size = "15px"
fare_fig.yaxis.major_label_text_font_size = "15px"

# Make the fare plot's yaxis tick labels indicate that these are dollar amounts
fare_fig.yaxis[0].formatter = NumeralTickFormatter(format="$0")

# To each plot add horizontal bars for mean & median, to discern a "typical month"
count_fig.line(x=count_by_month.index, y=(count_by_month.ride_count.mean()/1000), 
               color="green", line_dash=[2,2], legend_label='mean')
count_fig.line(x=count_by_month.index, y=(count_by_month.ride_count.median()/1000), 
               color="orange", line_dash=[8,8], legend_label='median')
fare_fig.line(x=fare_by_month.index, y=(fare_by_month.fare_amount.mean()/1000), 
               color="green", line_dash=[2,2])
fare_fig.line(x=fare_by_month.index, y=(fare_by_month.fare_amount.median()/1000), 
               color="orange", line_dash=[8,8])

# Create a legend
count_fig.legend.location = "top_left"
count_fig.legend.title = "Summary"
count_fig.legend.border_line_width = 3

# Establish grid
spacer = Spacer(width=25)
full_figure = grid(children=row(count_fig, spacer, fare_fig))

# show the results
output_notebook()
show(full_figure)

Loading BokehJS ... 
 
 Warning: Requesting CDN BokehJS version '3.1.0rc1' from local development version '3.1.0rc1+17.g3416314a'. This configuration is unsupported and may not work!

It looks like March-May is the range where ridership matches the mean/median. I might zero in on May for further analysis. The mean and median values for rides across the year aren't grossly different, and the patterns for ride volume and fares appear to closely co-vary. This is predictable, but I'm glad I confirmed. I'd be curious whether there are any variabes that would lead there to be a drop in fares relative to ride volume. 

Moving forward, because my ultimate aim is to learn about complex ways to visualize longer-term trends year-over-year, I'll focus on the following questions:

General question:
How have green taxi base fare revenues changed over time? 
Specific questions: 
- How might we predict future revenues? 
- What factors may be correlated with any changes?
- Since one objective for a taxi company is likely maximizing per-trip revenue, are there any times when fares dropped relative to ride volume, and if so, what other variables may have been associated with this change?

But first, I want to learn how to add summative titles to layouts. I'll work to create a minimal reproducible example here to share with others in order to solicit support with that. 

In [15]:
# Code that's similar to what I have now.

# Create simple data for MRE
x = [1,2,3,4,5]
y1 = [7.5, 6.2, 8.4, 8.7, 8.9]
y2 = [7.5, 6.9, 8.4, 8.9, 8.7]

# Prepare figures
p1 = figure(title = "Full Title for Figure 1", width=300, height=300)
p1.line(x=x, y=y1)

p2 = figure(title = "Full Title for Figure 2", width=300, height=300)
p2.line(x=x, y=y2)

# Establish grid
spacer = Spacer(width=5)
minimal_grid = grid(children=row(p1, spacer, p2))

# show the results
output_notebook()
show(minimal_grid)

Loading BokehJS ... 
 
 Warning: Requesting CDN BokehJS version '3.1.0rc1' from local development version '3.1.0rc1+17.g3416314a'. This configuration is unsupported and may not work!

In [16]:
# Code for moving toward what I want. 

# Code that's similar to what I have now.

# Create simple data for MRE
x = [1,2,3,4,5]
y1 = [7.5, 6.2, 8.4, 8.7, 8.9]
y2 = [7.5, 6.9, 8.4, 8.9, 8.7]

# Prepare figures
p1 = figure(title = "Subtitle for Figure 1", width=300, height=300)
p1.line(x=x, y=y1)

p2 = figure(title = "Subtitle for Figure 2", width=300, height=300)
p2.line(x=x, y=y2)

# Establish grid
spacer = Spacer(width=5)
minimal_grid = grid(children=row(p1, spacer, p2))

# show the results
output_notebook()
show(minimal_grid)

Loading BokehJS ... 
 
 Warning: Requesting CDN BokehJS version '3.1.0rc1' from local development version '3.1.0rc1+17.g3416314a'. This configuration is unsupported and may not work!